In [1]:
import re
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt

In [2]:
def select_gpu(N):
    gpus = tf.config.experimental.list_physical_devices('GPU')
    print(gpus)
    if gpus:
        try:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            tf.config.experimental.set_visible_devices(gpus[N], 'GPU')
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        except RuntimeError as e:
            print(e)    

In [3]:
select_gpu(0)
epochs = 4096
batch_size = 1024
model_name = 'bert-chinese'
embedding_path = '/home/Danny/pretrain_model/{}'.format(model_name)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU')]
3 Physical GPUs, 1 Logical GPUs


In [4]:
# 不用儲存article_id, 轉成list
def training_txt_to_list(path):
    with open(path, 'r') as f:
        txt = str(f.read())
    txt_list = txt.split('\n')
    text_label_list = list()
    tmp = list()
    for line in txt_list:
        if line == '--------------------':
            text_label_list.append(tmp)
            tmp = list()
            continue
        if line == '':
            continue
        tmp.append(line)
    x = list()
    y = list()
    for text_label in text_label_list:
        text = text_label[0]
        label = text_label[2:]
        label_list = ['O' for i in range(len(text))]
        for i in label:
            entity = i.split('\t')
            if int(entity[1]) > int(entity[2]):
                continue
            b = int(entity[1])
            label_list[b] = 'B-{}'.format(entity[-1])
            for j in range(int(entity[1])+1, int(entity[2])):
                label_list[j] = 'I-{}'.format(entity[-1])
        for i, j in enumerate(text):
            if j == '，' or j == '。' or j == '？':
                label_list[i] = j
                
        text_list = re.split('\uff0c|\u3002|\uff1f', text)
        for sentence in text_list:
            x.append([i for i in sentence])
        x = x[:-1]
            
        sentence = list()
        for i in label_list:
            if i == '，' or i == '。' or i == '？':
                y.append(sentence)
                sentence = list()
            else:
                sentence.append(i)
                
    return [x, y]

In [5]:
x1, y1 = training_txt_to_list('/home/Danny/AI-CUP-2020/datasets/stage1/SampleData_deid.txt')

In [6]:
x2, y2 = training_txt_to_list('/home/Danny/AI-CUP-2020/datasets/stage2/train_1_update.txt')

In [7]:
x4, y4 = training_txt_to_list('/home/Danny/AI-CUP-2020/datasets/stage4/train_2.txt')

In [8]:
x = x1 + x2 + x4
y = y1 + y2 + y4
print(len(x))
print(len(y))

76162
76162


In [9]:
df = pd.DataFrame({'x': x, 'y': y})
df

,x,y
0,"[醫, 師, ：, 你, 有, 做, 超, 音, 波, 嘛]","[O, O, O, O, O, O, O, O, O, O]"
1,"[那, 我, 們, 來, 看, 報, 告]","[O, O, O, O, O, O, O]"
2,"[有, 些, 部, 分, 有, 紅, 字, 耶]","[O, O, O, O, O, O, O, O]"
3,"[民, 眾, ：, 紅, 字, 是, 甚, 麼, 意, 思]","[O, O, O, O, O, O, O, O, O, O]"
4,"[醫, 師, ：, 就, 是, 肝, 功, 能, 有, 比, 較, 高]","[O, O, O, O, O, O, O, O, O, O, O, O]"
...,...,...
76157,"[醫, 師, ：, 他, 有, 在, 騎, 腳, 踏, 車]","[O, O, O, O, O, O, O, O, O, O]"
76158,"[民, 眾, ：, 恩]","[O, O, O, O]"
76159,"[醫, 師, ：, 騎, 腳, 踏, 車, 可, 以]","[O, O, O, O, O, O, O, O, O]"
76160,"[小, 美]","[B-name, I-name]"


In [10]:
df['x'] = df['x'].apply(lambda row : str(row))
df['y'] = df['y'].apply(lambda row : str(row))
df = df.drop_duplicates()
df['x'] = df['x'].apply(lambda row : eval(row))
df['y'] = df['y'].apply(lambda row : eval(row))
df

/home/Danny/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/Danny/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,x,y
0,"[醫, 師, ：, 你, 有, 做, 超, 音, 波, 嘛]","[O, O, O, O, O, O, O, O, O, O]"
1,"[那, 我, 們, 來, 看, 報, 告]","[O, O, O, O, O, O, O]"
2,"[有, 些, 部, 分, 有, 紅, 字, 耶]","[O, O, O, O, O, O, O, O]"
3,"[民, 眾, ：, 紅, 字, 是, 甚, 麼, 意, 思]","[O, O, O, O, O, O, O, O, O, O]"
4,"[醫, 師, ：, 就, 是, 肝, 功, 能, 有, 比, 較, 高]","[O, O, O, O, O, O, O, O, O, O, O, O]"
...,...,...
75713,"[醫, 師, ：, 7, 4, 0, 3]","[O, O, O, B-ID, I-ID, I-ID, I-ID]"
75714,"[醫, 師, ：, 6, 6, 6, 0, 8, 9, 7, 4, 0, 3]","[O, O, O, B-ID, I-ID, I-ID, I-ID, I-ID, I-ID, ..."
75716,"[護, 理, 師, ：, 6, 6, 6, 0, 9, 8]","[O, O, O, O, B-ID, I-ID, I-ID, I-ID, I-ID, I-ID]"
75717,"[醫, 師, ：, 0, 9, 8, 7, 4, 0, 3]","[O, O, O, B-ID, I-ID, I-ID, I-ID, I-ID, I-ID, ..."


In [11]:
x = df['x'].to_list()
y = df['y'].to_list()

In [12]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2, random_state=42)
train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size=0.2, random_state=42)
print(len(train_x), len(train_y))
print(len(valid_x), len(valid_y))
print(len(test_x), len(test_y))

20172 20172
5044 5044
6305 6305


In [ ]:
%%time
import kashgari
from kashgari.tasks.labeling import BiLSTM_CRF_Model
from kashgari.embeddings import BertEmbedding
kashgari.config.use_cudnn_cell = True
bert_embedding = BertEmbedding(embedding_path, 
                           sequence_length='auto',
                           trainable=True,
                           task='kashgari.LABELING', 
                          )
model = BiLSTM_CRF_Model(bert_embedding)
history = model.fit(train_x,
                    train_y,
                    valid_x,
                    valid_y,
                    epochs=epochs,
                    batch_size=batch_size,
                   )

2020-12-15 19:50:42,095 [DEBUG] kashgari - ------------------------------------------------
2020-12-15 19:50:42,096 [DEBUG] kashgari - Loaded transformer model's vocab
2020-12-15 19:50:42,097 [DEBUG] kashgari - config_path       : /home/Danny/pretrain_model/bert-chinese/bert_config.json
2020-12-15 19:50:42,097 [DEBUG] kashgari - vocab_path      : /home/Danny/pretrain_model/bert-chinese/vocab.txt
2020-12-15 19:50:42,098 [DEBUG] kashgari - checkpoint_path : /home/Danny/pretrain_model/bert-chinese/bert_model.ckpt
2020-12-15 19:50:42,098 [DEBUG] kashgari - Top 50 words    : ['[PAD]', '[unused1]', '[unused2]', '[unused3]', '[unused4]', '[unused5]', '[unused6]', '[unused7]', '[unused8]', '[unused9]', '[unused10]', '[unused11]', '[unused12]', '[unused13]', '[unused14]', '[unused15]', '[unused16]', '[unused17]', '[unused18]', '[unused19]', '[unused20]', '[unused21]', '[unused22]', '[unused23]', '[unused24]', '[unused25]', '[unused26]', '[unused27]', '[unused28]', '[unused29]', '[unused30]', '[

Epoch 1/4096
19/19 [==============================] - 17s 919ms/step - loss: 4.4900 - accuracy: 0.8513 - val_loss: 28.7340 - val_accuracy: 0.9696
Epoch 2/4096
19/19 [==============================] - 13s 698ms/step - loss: 1.2163 - accuracy: 0.9659 - val_loss: 28.6359 - val_accuracy: 0.9716
Epoch 3/4096
19/19 [==============================] - 13s 694ms/step - loss: 0.8308 - accuracy: 0.9709 - val_loss: 28.5179 - val_accuracy: 0.9752
Epoch 4/4096
19/19 [==============================] - 13s 695ms/step - loss: 0.6781 - accuracy: 0.9740 - val_loss: 28.4310 - val_accuracy: 0.9781
Epoch 5/4096
19/19 [==============================] - 13s 696ms/step - loss: 0.5777 - accuracy: 0.9763 - val_loss: 28.4485 - val_accuracy: 0.9793
Epoch 6/4096
19/19 [==============================] - 13s 698ms/step - loss: 0.5138 - accuracy: 0.9778 - val_loss: 28.4729 - val_accuracy: 0.9804
Epoch 7/4096
19/19 [==============================] - 13s 699ms/step - loss: 0.4590 - accuracy: 0.9792 - val_loss: 28.2088 -

Epoch 57/4096
19/19 [==============================] - 13s 705ms/step - loss: 0.0632 - accuracy: 0.9965 - val_loss: 27.2404 - val_accuracy: 0.9846
Epoch 58/4096
19/19 [==============================] - 13s 705ms/step - loss: 0.0618 - accuracy: 0.9964 - val_loss: 27.2892 - val_accuracy: 0.9846
Epoch 59/4096
19/19 [==============================] - 13s 703ms/step - loss: 0.0581 - accuracy: 0.9965 - val_loss: 27.3067 - val_accuracy: 0.9850
Epoch 60/4096
19/19 [==============================] - 13s 705ms/step - loss: 0.0583 - accuracy: 0.9965 - val_loss: 27.2309 - val_accuracy: 0.9854
Epoch 61/4096
19/19 [==============================] - 14s 714ms/step - loss: 0.0574 - accuracy: 0.9967 - val_loss: 27.3725 - val_accuracy: 0.9852
Epoch 62/4096
19/19 [==============================] - 14s 714ms/step - loss: 0.0560 - accuracy: 0.9968 - val_loss: 27.1875 - val_accuracy: 0.9851
Epoch 63/4096
19/19 [==============================] - 13s 705ms/step - loss: 0.0531 - accuracy: 0.9970 - val_loss: 27

19/19 [==============================] - 13s 705ms/step - loss: 0.0236 - accuracy: 0.9984 - val_loss: 26.6753 - val_accuracy: 0.9831
Epoch 113/4096
19/19 [==============================] - 13s 704ms/step - loss: 0.0220 - accuracy: 0.9986 - val_loss: 26.7163 - val_accuracy: 0.9830
Epoch 114/4096
19/19 [==============================] - 13s 705ms/step - loss: 0.0224 - accuracy: 0.9987 - val_loss: 26.7960 - val_accuracy: 0.9833
Epoch 115/4096
19/19 [==============================] - 13s 704ms/step - loss: 0.0210 - accuracy: 0.9987 - val_loss: 26.7654 - val_accuracy: 0.9833
Epoch 116/4096
19/19 [==============================] - 13s 704ms/step - loss: 0.0206 - accuracy: 0.9986 - val_loss: 26.7949 - val_accuracy: 0.9835
Epoch 117/4096
19/19 [==============================] - 13s 707ms/step - loss: 0.0228 - accuracy: 0.9985 - val_loss: 26.7718 - val_accuracy: 0.9837
Epoch 118/4096
19/19 [==============================] - 13s 705ms/step - loss: 0.0218 - accuracy: 0.9986 - val_loss: 26.7425 - 

19/19 [==============================] - 13s 703ms/step - loss: 0.0145 - accuracy: 0.9989 - val_loss: 26.3661 - val_accuracy: 0.9817
Epoch 168/4096
19/19 [==============================] - 13s 703ms/step - loss: 0.0150 - accuracy: 0.9989 - val_loss: 26.4061 - val_accuracy: 0.9833
Epoch 169/4096
19/19 [==============================] - 13s 703ms/step - loss: 0.0157 - accuracy: 0.9987 - val_loss: 26.3445 - val_accuracy: 0.9820
Epoch 170/4096
19/19 [==============================] - 13s 702ms/step - loss: 0.0165 - accuracy: 0.9989 - val_loss: 26.3225 - val_accuracy: 0.9810
Epoch 171/4096
19/19 [==============================] - 13s 703ms/step - loss: 0.0164 - accuracy: 0.9989 - val_loss: 26.4221 - val_accuracy: 0.9822
Epoch 172/4096
19/19 [==============================] - 14s 720ms/step - loss: 0.0156 - accuracy: 0.9988 - val_loss: 26.3407 - val_accuracy: 0.9821
Epoch 173/4096
19/19 [==============================] - 13s 704ms/step - loss: 0.0146 - accuracy: 0.9990 - val_loss: 26.2792 - 

...

19/19 [==============================] - 14s 713ms/step - loss: 0.0043 - accuracy: 0.9992 - val_loss: 19.1232 - val_accuracy: 0.0057
Epoch 1103/4096
19/19 [==============================] - 13s 704ms/step - loss: 0.0043 - accuracy: 0.9992 - val_loss: 19.1048 - val_accuracy: 0.0055
Epoch 1104/4096
19/19 [==============================] - 13s 704ms/step - loss: 0.0044 - accuracy: 0.9991 - val_loss: 19.1448 - val_accuracy: 0.0055
Epoch 1105/4096
19/19 [==============================] - 13s 704ms/step - loss: 0.0044 - accuracy: 0.9992 - val_loss: 19.0837 - val_accuracy: 0.0053
Epoch 1106/4096
19/19 [==============================] - 13s 705ms/step - loss: 0.0039 - accuracy: 0.9993 - val_loss: 19.0210 - val_accuracy: 0.0052
Epoch 1107/4096
19/19 [==============================] - 13s 704ms/step - loss: 0.0048 - accuracy: 0.9991 - val_loss: 19.1602 - val_accuracy: 0.0050
Epoch 1108/4096
19/19 [==============================] - 13s 705ms/step - loss: 0.0043 - accuracy: 0.9992 - val_loss: 19.1

19/19 [==============================] - 13s 704ms/step - loss: 0.0045 - accuracy: 0.9991 - val_loss: 18.9496 - val_accuracy: 0.0043
Epoch 1158/4096
19/19 [==============================] - 13s 701ms/step - loss: 0.0051 - accuracy: 0.9990 - val_loss: 18.9812 - val_accuracy: 0.0054
Epoch 1159/4096
19/19 [==============================] - 13s 703ms/step - loss: 0.0051 - accuracy: 0.9989 - val_loss: 19.0301 - val_accuracy: 0.0049
Epoch 1160/4096
19/19 [==============================] - 13s 703ms/step - loss: 0.0040 - accuracy: 0.9992 - val_loss: 19.0700 - val_accuracy: 0.0047
Epoch 1161/4096
19/19 [==============================] - 14s 714ms/step - loss: 0.0045 - accuracy: 0.9990 - val_loss: 19.0299 - val_accuracy: 0.0046
Epoch 1162/4096
19/19 [==============================] - 14s 713ms/step - loss: 0.0041 - accuracy: 0.9991 - val_loss: 19.0732 - val_accuracy: 0.0050
Epoch 1163/4096
19/19 [==============================] - 13s 704ms/step - loss: 0.0044 - accuracy: 0.9991 - val_loss: 18.9

19/19 [==============================] - 13s 703ms/step - loss: 0.0045 - accuracy: 0.9991 - val_loss: 18.9613 - val_accuracy: 0.0036
Epoch 1213/4096
19/19 [==============================] - 13s 703ms/step - loss: 0.0045 - accuracy: 0.9990 - val_loss: 18.9113 - val_accuracy: 0.0037
Epoch 1214/4096
19/19 [==============================] - 13s 704ms/step - loss: 0.0043 - accuracy: 0.9991 - val_loss: 18.8461 - val_accuracy: 0.0043
Epoch 1215/4096
19/19 [==============================] - 13s 702ms/step - loss: 0.0043 - accuracy: 0.9990 - val_loss: 18.8562 - val_accuracy: 0.0039
Epoch 1216/4096
19/19 [==============================] - 13s 704ms/step - loss: 0.0050 - accuracy: 0.9991 - val_loss: 18.8508 - val_accuracy: 0.0042
Epoch 1217/4096
19/19 [==============================] - 13s 703ms/step - loss: 0.0049 - accuracy: 0.9990 - val_loss: 18.9783 - val_accuracy: 0.0043
Epoch 1218/4096
19/19 [==============================] - 13s 702ms/step - loss: 0.0051 - accuracy: 0.9990 - val_loss: 18.9

19/19 [==============================] - 13s 704ms/step - loss: 0.0042 - accuracy: 0.9990 - val_loss: 18.8158 - val_accuracy: 0.0034
Epoch 1268/4096
19/19 [==============================] - 13s 703ms/step - loss: 0.0042 - accuracy: 0.9990 - val_loss: 18.8398 - val_accuracy: 0.0032
Epoch 1269/4096
19/19 [==============================] - 13s 703ms/step - loss: 0.0040 - accuracy: 0.9991 - val_loss: 18.8064 - val_accuracy: 0.0033
Epoch 1270/4096
19/19 [==============================] - 13s 704ms/step - loss: 0.0042 - accuracy: 0.9991 - val_loss: 18.8362 - val_accuracy: 0.0031
Epoch 1271/4096
19/19 [==============================] - 13s 704ms/step - loss: 0.0041 - accuracy: 0.9991 - val_loss: 18.8746 - val_accuracy: 0.0033
Epoch 1272/4096
19/19 [==============================] - 13s 705ms/step - loss: 0.0041 - accuracy: 0.9992 - val_loss: 18.8016 - val_accuracy: 0.0038
Epoch 1273/4096
19/19 [==============================] - 13s 703ms/step - loss: 0.0035 - accuracy: 0.9992 - val_loss: 18.8

19/19 [==============================] - 14s 713ms/step - loss: 0.0039 - accuracy: 0.9992 - val_loss: 18.7716 - val_accuracy: 0.0036
Epoch 1323/4096
19/19 [==============================] - 13s 703ms/step - loss: 0.0045 - accuracy: 0.9990 - val_loss: 18.7545 - val_accuracy: 0.0033
Epoch 1324/4096
19/19 [==============================] - 13s 702ms/step - loss: 0.0039 - accuracy: 0.9991 - val_loss: 18.7936 - val_accuracy: 0.0031
Epoch 1325/4096
19/19 [==============================] - 13s 704ms/step - loss: 0.0048 - accuracy: 0.9990 - val_loss: 18.7318 - val_accuracy: 0.0035
Epoch 1326/4096
19/19 [==============================] - 13s 704ms/step - loss: 0.0044 - accuracy: 0.9991 - val_loss: 18.7791 - val_accuracy: 0.0033
Epoch 1327/4096
19/19 [==============================] - 13s 702ms/step - loss: 0.0053 - accuracy: 0.9992 - val_loss: 18.7022 - val_accuracy: 0.0033
Epoch 1328/4096
19/19 [==============================] - 13s 702ms/step - loss: 0.0038 - accuracy: 0.9993 - val_loss: 18.7

In [ ]:
model.evaluate(test_x, test_y)
model_path = 'model/{}_epoch_{}_batch_{}'.format(model_name, epochs, batch_size)
model.save(model_path)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['accuracy', 'val_accuracy', 'loss', 'val_loss'])
plt.show()

In [ ]:
# 需要儲存article_id, 轉成dataframe
def predicting_txt_to_dataframe(path):    
    with open(path, 'r') as f:
        txt = str(f.read())
        txt_list = txt.split('\n')
    row_list = list()
    tmp_list = list()
    for row in txt_list:
        if row == '--------------------':
            tmp_list[0] = tmp_list[0].replace('article_id:', '')
            tmp_list[0] = int(tmp_list[0])
            row_list.append(tmp_list)
            tmp_list = list()
            continue
        if len(row) > 0:
            tmp_list.append(row)
    df = pd.DataFrame(row_list, columns=['article_id','text'])
    return df

In [ ]:
def predict_NER(text):
    x_list = list()    
    text_list = re.split('\uff0c|\u3002|\uff1f', text)
    for article in text_list:
        x_list.append([i for i in article])
    y_list_list = model.predict(x_list)
    y_list = list()
    for sentence in y_list_list:
        for word in sentence:
            y_list.append(word)
        y_list.append('O') # append(，。？)
    y_list = y_list[:-1]
    return y_list

In [ ]:
def output_NER(article_id, text, y_list):
    output_str = str()
    flag = False
    for i, j in enumerate(y_list):
        if j != 'O':
            if j[0] == 'B':
                start_position = i
                entity_type = j.split('-')[-1]
                flag = True
        elif j == 'O' and flag == True:
            end_position = i
            flag = False
            entity_text = text[start_position: end_position]
            entity = '{}\t{}\t{}\t{}\t{}\n'.format(article_id, start_position, end_position, entity_text, entity_type)  
            output_str += entity
    return output_str

In [ ]:
df = predicting_txt_to_dataframe('/home/Danny/AI-CUP-2020/datasets/stage5/test.txt')
output_str = "article_id\tstart_position\tend_position\tentity_text\tentity_type\n"

for article_id, text in zip(df['article_id'], df['text']):
    x_list = [word for word in text]
    y_list = predict_NER(text)
    output_str += output_NER(article_id, text, y_list)

In [ ]:
output_path = 'output/{}_epoch_{}_batch_{}.tsv'.format(model_name, epochs, batch_size)
with open(output_path, 'w', encoding='utf-8') as f:
    f.write(output_str)

In [ ]:
df = pd.read_csv(output_path, sep='\t')
df